In [2]:
!python --version
#!which python
#!pip install torch torchvision torchaudio << didn't work, CUDA 12.1

# use this for CUDA 11.8 (on yeti)
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Python 3.8.10


In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
# >>> Cora()
# len(dataset)
# >>> 1

# dataset.num_classes
# >>> 7

# dataset.num_node_features
# >>> 1433

In [5]:
# number of graphs in dataset
len(dataset)

1

In [6]:
dataset.num_features

1433

In [7]:
dataset.num_edge_features

0

In [8]:
dataset[0].num_node_features

1433

In [9]:
dataset[0].num_nodes

2708

In [10]:
# [num nodes, num node features]
dataset[0].x.shape

torch.Size([2708, 1433])